Unificando el scraper

In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Image
import pandas as pd

In [2]:
link_principal='https://www.pagina12.com.ar'
### editando cabera para evitar el error 429 
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36"}
###
r_web_principal=requests.get(link_principal, headers=headers)
s_web_principal = BeautifulSoup(r_web_principal.text, 'lxml')

In [3]:
#seleccionamos el primer elemento de la lista de secciones
secciones_web_principal = s_web_principal.find_all('div', attrs={'class':'p12-dropdown-column'})[0]
link_secciones_web_principal = secciones_web_principal.find_all('a')
#recorremos la lista de tag a
URL_secciones=[]
for i in link_secciones_web_principal:
    if len(link_secciones_web_principal):
        URL_secciones.append(i.get('href'))

In [4]:
r_URL_secciones= requests.get(URL_secciones[0])
s_seccion= BeautifulSoup(r_URL_secciones.text, 'lxml')
def obtener_noticias(soup):
    lista_notas=[]
    #obtener noticias promocionadas
    noticias_promocionadas=soup.find('div', attrs={'class':'article-item__content'})
    if noticias_promocionadas:
        lista_notas.append(f'{link_principal}{noticias_promocionadas.a.get("href")}')

    #resto de noticias
    resto_noticias=soup.find('section', attrs={'class':'list-content'})
    for i in resto_noticias.find_all('div', attrs={'class':'article-item__header'}):
        if i.a:
            lista_notas.append(f'{link_principal}{i.a.get("href")}')

    return lista_notas

In [5]:
lista_notas=(obtener_noticias(s_seccion))

In [6]:
# obtemos informacion de cada nota
# seleccionamos la primera nota de nuestra lista de notas
URL_nota=lista_notas[0]
r_URL_nota= requests.get(URL_nota)
s_nota=BeautifulSoup(r_URL_nota.text, 'lxml')

def obtener_info(s_nota):
    #creamos un diccionario vacio para guardar informacion de la notas
    info_dic={}

    #extraemos el titulo de la nota
    titulo = s_nota.find('h1').text
    if titulo:
        info_dic['titulo']=titulo
    else:
        info_dic['titulo']=None

    #extraemos la fecha de la nota
    fecha = s_nota.find('time').get_text()
    if fecha:
        info_dic['fecha']=fecha
    else:
        info_dic['fecha']=None

    #extraemos el copete
    copete= s_nota.find('h4').get_text()
    if copete:
        info_dic['copete']=copete
    else:
        info_dic['copete']=None

    #extraemos la volanta
    volanta= s_nota.find('h3').get_text()
    if volanta:
        info_dic['volanta']=volanta
    else:
        info_dic['volanta']=None

    #extraemos el cuerpo de la nota
    cuerpo= s_nota.find('div', attrs={'class':'article-main-content article-text'}).find_all('p')
    cu=[]
    for i in cuerpo:
        cu.append(i.get_text())
    
    #utilizamos el metodo join, para convertir una lista str a una cadena de texto
    if cu:
        info_dic['cuerpo']="".join(map(str, cu))
    else:
        info_dic['cuerpo']=None
    
    #extraemos el autor de la nota
    autor= s_nota.find('div', attrs={'class':'author-name'})
    if autor:
        info_dic['autor']=autor.get_text()
    else:
        info_dic['autor']=None
    
    #extraemos la imagen
    #parseamos el tag div que contiene la imagen de la nota
    media = s_nota.find('div', attrs={'class':'article-main-media-image__container'})
    if len(media):
        imagen_src=media.img.get('src')
        try:
            r_img=requests.get(imagen_src)
            if r_img.status_code== 200:
                info_dic['imagen']=r_img.content
            else:
                info_dic['imagen']=None
        except:
            print('No se pudo obtener la imagen')
    else:
        print ('No se encontraron imagenes')

    return  info_dic
#obtiene todos los valores de un diccionario
#print (dic.values())
#obtener un valor de un diccionario, utilizamos la libreria de imagen de python para mostrar la img
#Image(dic.get('imagen'))

In [7]:
# creamos nuestra funcion para recorrer cada noticia de las secciones
# de esta manera poder obtener la informacion de cada noticia
# y devolvemos un diccionario de todos los datos de la noticia
def scrape_nota(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36"} 
        r_nota=requests.get(url, headers=headers)
    except Exception as e:
        print ('Error scrapeando URL ->', url)
        print (e)
        return None
    
    if r_nota.status_code!= 200:
        print(f'Error obteniendo nota {url}')
        print(f'Status Code = {r_nota.status_code}')
        return None
    
    s_nota=BeautifulSoup(r_nota.text, 'lxml')

    dic_scrape=obtener_info(s_nota)
    dic_scrape['url']=url

    return dic_scrape

In [8]:
URL_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/edicion-impresa',
 'https://www.pagina12.com.ar/secciones/universidad-diario',
 'https://www.pagina12.com.ar/secciones/ajedrez',
 'https://www.pagina12.com.ar/secciones/cultura',
 'https://www.pagina12.com.ar/secciones/dialogos',
 'https://www.pagina12.com.ar/secciones/plastica',
 'https://www.pagina12.com.ar/secciones/psicologia',
 'https://www.pagina12.com.ar/secciones/cartas-de-lectores',
 'https://www.pagina12.com.ar/secciones/contratapa',
 'https://www.pagina12.com.ar/secciones/audiovisuales',
 'https://www.pagina12.com.ar/secciones/recordatorios',
 'https://www.pagina12.com.ar/secciones

In [9]:
#obtenemos todos los link de la primera pagina de cada seccion 
notas_secciones=[]
for link in URL_secciones:
    try:
        r_2_URL_secciones=requests.get(link)
        if r_2_URL_secciones.status_code== 200:
            #parseamos cada seccion 
            s_2_web_principal=BeautifulSoup(r_2_URL_secciones.text, 'lxml')
            # enviamos el link de la seccion a nuestra funcion obtener notas 
            # y guaramos lo que retorna la funcion (diccionario) a nuestra lista de notas de secciones
            notas_secciones.extend(obtener_noticias(s_2_web_principal))
        else:
            print ('No se pudo obtener la seccion ->', link)
    except:
        print ('No se pudo obtener la seccion ->', link)

No se pudo obtener la seccion -> https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos
No se pudo obtener la seccion -> https://www.pagina12.com.ar/edicion-impresa


In [10]:
print(notas_secciones)

['https://www.pagina12.com.ar/495725-nueva-audiencia-en-la-insolita-causa-del-memorandum', 'https://www.pagina12.com.ar/495524-la-justicia-en-modo-macri', 'https://www.pagina12.com.ar/495590-cristina-kirchner-recomendo-en-sus-redes-sociales-una-nota-p', 'https://www.pagina12.com.ar/495662-comenzo-el-juicio-por-la-apropiacion-de-victoria-donda', 'https://www.pagina12.com.ar/495670-el-abrazo-entre-peron-y-balbin', 'https://www.pagina12.com.ar/495708-la-familia-de-santiago-maldonado-ya-no-participara-en-la-cau', 'https://www.pagina12.com.ar/495709-sobre-el-presupuesto-educativo-en-caba', 'https://www.pagina12.com.ar/495710-martes-de-superaccion-en-juntos-por-el-cambio', 'https://www.pagina12.com.ar/495713-caba-la-crueldad-de-la-derecha-al-desnudo', 'https://www.pagina12.com.ar/495716-oscar-ojea-la-division-de-la-sociedad-es-motivo-de-escandalo', 'https://www.pagina12.com.ar/495735-pablo-moyano-a-kelly-olmos-es-bono-o-suma-fija-de-forma-urge', 'https://www.pagina12.com.ar/495736-los-coleta

In [15]:
# creamos nuestro data set de noticias de cada seccion
data_set=[]
# recorremos cada noticia de cada seccion
# en cada noticia utilizamos el metodo scrape_nota que a su vez llama a obtener_info
# con el resultado de obtener_info, vamos guardando en un diccionario -> data_set
for posicion_nota,link_nota in enumerate(notas_secciones):
    if posicion_nota<50:
        print (f'Scrapeando nota {posicion_nota} / {len(link_nota)}')
        data_set.append(scrape_nota(link_nota))

Scrapeando nota 0 / 86
Scrapeando nota 1 / 60
Scrapeando nota 2 / 95
Scrapeando nota 3 / 89
Scrapeando nota 4 / 65
Scrapeando nota 5 / 95
Scrapeando nota 6 / 73
Scrapeando nota 7 / 80
Scrapeando nota 8 / 76
Scrapeando nota 9 / 95
Scrapeando nota 10 / 95
Scrapeando nota 11 / 84
Scrapeando nota 12 / 77
Scrapeando nota 13 / 76
Scrapeando nota 14 / 95
Scrapeando nota 15 / 72
Scrapeando nota 16 / 67
Scrapeando nota 17 / 58
Scrapeando nota 18 / 95
Scrapeando nota 19 / 95
Scrapeando nota 20 / 65
Scrapeando nota 21 / 56
Scrapeando nota 22 / 95
Scrapeando nota 23 / 79
Scrapeando nota 24 / 95
Scrapeando nota 25 / 49
Scrapeando nota 26 / 94
Scrapeando nota 27 / 95
Scrapeando nota 28 / 95
Scrapeando nota 29 / 95
Scrapeando nota 30 / 95
Scrapeando nota 31 / 76
Scrapeando nota 32 / 95
Scrapeando nota 33 / 95
Scrapeando nota 34 / 95
Scrapeando nota 35 / 95
Scrapeando nota 36 / 95
Scrapeando nota 37 / 95
Scrapeando nota 38 / 95
Scrapeando nota 39 / 95
Scrapeando nota 40 / 84
Scrapeando nota 41 / 95
Sc

In [16]:
# creamos nuestro data frame utilizando pandas data frame
data_frame= pd.DataFrame(data_set)
#podemos visualizar con las cabeceras
data_frame.head()
# pasamos nuestro conjunto de datos a un archivo csv
data_frame.to_csv('Notas pagina12.csv')
# pasamos nuestro conjunto de datos a un archivo excel
data_frame.to_excel("Notas Paginas12.xlsx", sheet_name="data frame", index=True)